# Forecasting Stock Prices using Prophet

- Forecasting is a hard science and requires substantial expertise. For these reasons Facebook open-sourced Prophet, which allows anyone with Python programming skills to implement forecasts on time series easily. You can read more about Prophet on Facebook’s blog or in the paper.
- In this article, we will experiment with using Prophet to forecast stock prices. Prophet uses a decomposable time series model with three main model components: growth, seasonality and holidays. They are combined using the equation

##### y(t) = g(t) + s(t) + h(t) + e(t),

* https://github.com/NGYB/Stocks/blob/master/StockPricePrediction_fh21/StockPricePrediction_v2_prophet.ipynb
* https://towardsdatascience.com/forecasting-stock-prices-using-prophet-652b31fb564e

## Objective:
- 2019-02-13:
- Given prices for the last N days, we do prediction for the next N+H days, where H is the forecast horizon
- Instead of using last value method, we use prophet package to predict

In [ ]:
%matplotlib inline
import math
import matplotlib
import multiprocessing
import numpy as np
import pandas as pd
import pickle
import seaborn as sns
import time

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from datetime import date, datetime, timedelta
from fbprophet import Prophet
from joblib import Parallel, delayed
from matplotlib import pyplot as plt
from pylab import rcParams
from sklearn.metrics import mean_squared_error
from tqdm import tqdm_notebook

#### Input params ##################
stk_path = r'C:/Users/Gordon D. Pisciotta/Stocks/StockPricePrediction_fh21/data/VTI_20130102_20181231.csv'
H = 21                          # Forecast horizon
train_size = 252*3              # Use 3 years of data as train set. Note there are about 252 trading days in a year
val_size = 252                  # Use 1 year of data as validation set
changepoint_prior_scale_list = [0.05, 0.5, 1, 1.5, 2.5]     # for hyperparameter tuning
fourier_order_list = [None, 2, 4, 6, 8, 10]                 # for hyperparameter tuning
holidays_path = r'C:/Users/Gordon D. Pisciotta/Stocks/StockPricePrediction_fh21/data/holidays.csv'
window_list = [None, 0, 1, 2]                               # for hyperparameter tuning

fontsize = 14
ticklabelsize = 14
####################################

train_val_size = train_size + val_size # Size of train+validation set
print("No. of days in train+validation set = " + str(train_val_size))

In [ ]:
print("We will start forecasting on day %d" % (train_val_size+1))

## Common functions


In [ ]:
def get_mape(y_true, y_pred): 
    """
    Compute mean absolute percentage error (MAPE)
    """
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def get_mae(a, b):
    """
    Comp mean absolute error e_t = E[|a_t - b_t|]. a and b can be lists.
    Returns a vector of len = len(a) = len(b)
    """
    return np.mean(abs(np.array(a)-np.array(b)))

def get_rmse(a, b):
    """
    Comp RMSE. a and b can be lists.
    Returns a scalar.
    """
    return math.sqrt(np.mean((np.array(a)-np.array(b))**2))

def get_preds_prophet(df, H, changepoint_prior_scale=0.05, fourier_order=None, holidays=None):
    """
    Use Prophet to forecast for the next H timesteps, starting at df[len(df)]
    Inputs
        df: dataframe with headers 'ds' and 'y' (necessary for Prophet)
        H : forecast horizon
        changepoint_prior_scale : to detect changepoints in time series analysis trajectories
        fourier_order           : determines how quickly seasonality can change
        holidays                : dataframe containing holidays you will like to model. 
                                  Must have 'holiday' and 'ds' columns
    Outputs
        A list of predictions
    """
    # Fit prophet model
    if holidays is not None:
        m = Prophet(changepoint_prior_scale=changepoint_prior_scale, holidays=holidays)
    else:
        m = Prophet(changepoint_prior_scale=changepoint_prior_scale)
    if (fourier_order is not None) and (~np.isnan(fourier_order)): # add monthly seasonality
        m.add_seasonality(name='monthly', period=21, fourier_order=int(fourier_order))

    m.fit(df)
    
    # Make future dataframe
    future = m.make_future_dataframe(periods=2*H)
    
    # Eliminate weekend from future dataframe
    future['day'] = future['ds'].dt.weekday
    future = future[future['day']<=4]
    
    # Predict
    forecast = m.predict(future) # Note this prediction includes the original dates
    forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

    return forecast['yhat'][len(df):len(df)+H]
    
def get_error_metrics(df, H, train_size, val_size, changepoint_prior_scale=0.05, fourier_order=None, holidays=None):
    """
    Given a dataframe consisting of both train+validation, do predictions of forecast horizon H on the validation set, 
    at H/2 intervals.
    Inputs
        df                     : dataframe with headers 'ds' and 'y' (necessary for Prophet)
        H                      : forecast horizon
        train_size             : length of training set
        val_size               : length of validation set. Note len(df) = train_size + val_size
        changepoint_prior_scale: to detect changepoints in time series analysis trajectories
        fourier_order          : determines how quickly seasonality can change
        holidays               : dataframe containing holidays you will like to model. 
                                 Must have 'holiday' and 'ds' columns
    Outputs
        mean of rmse, mean of mape, mean of mae, dict of predictions
    """
    assert len(df) == train_size + val_size
    
    # Predict using Prophet, and compute error metrics also
    rmse = [] # root mean square error
    mape = [] # mean absolute percentage error
    mae = []  # mean absolute error
    preds_dict = {}
    
    rmse_mean, mape_mean, mae_mean = get_preds_prophet_parallelized(df, H, changepoint_prior_scale, fourier_order, holidays)

    return rmse_mean, mape_mean, mae_mean, _

def hyperparam_tune_cp(df, H, train_size, val_size, changepoint_prior_scale_list):
    """
    Hyperparameter tuning - changepoint
    Inputs
        df                     : dataframe with headers 'ds' and 'y' (necessary for Prophet)
        H                      : forecast horizon
        train_size             : length of training set
        val_size               : length of validation set. Note len(df) = train_size + val_size
        changepoint_prior_scale_list: list of changepoint_prior_scale values to try
    Outputs
        optimum hyperparameters
    """
    rmse_mean_list = []
    mape_mean_list = []
    mae_mean_list = []
    for changepoint_prior_scale in tqdm_notebook(changepoint_prior_scale_list):
        print("changepoint_prior_scale = " + str(changepoint_prior_scale))
        rmse_mean, mape_mean, mae_mean, _ = get_error_metrics(df, H, train_size, val_size, changepoint_prior_scale)
        rmse_mean_list.append(rmse_mean)
        mape_mean_list.append(mape_mean)
        mae_mean_list.append(mae_mean)
    
    # Create results dataframe
    results = pd.DataFrame({'changepoint_prior_scale': changepoint_prior_scale_list,
                            'rmse': rmse_mean_list,
                            'mape(%)': mape_mean_list,
                            'mae': mae_mean_list})
    
    # Return hyperparam corresponding to lowest error metric
    return changepoint_prior_scale_list[np.argmin(rmse_mean_list)], results

def hyperparam_tune_fo(df, H, train_size, val_size, fourier_order_list):
    """
    Hyperparameter tuning - fourier order
    Inputs
        df                     : dataframe with headers 'ds' and 'y' (necessary for Prophet)
        H                      : forecast horizon
        train_size             : length of training set
        val_size               : length of validation set. Note len(df) = train_size + val_size
        fourier_order_list     : list of fourier_order values to try
    Outputs
        optimum hyperparameters
    """
    rmse_mean_list = []
    mape_mean_list = []
    mae_mean_list = []
    for fourier_order in tqdm_notebook(fourier_order_list):
        print("fourier_order = " + str(fourier_order))
        rmse_mean, mape_mean, mae_mean, _ = get_error_metrics(df, H, train_size, val_size, 0.05, fourier_order)
        rmse_mean_list.append(rmse_mean)
        mape_mean_list.append(mape_mean)
        mae_mean_list.append(mae_mean)
        
    # Create results dataframe
    results = pd.DataFrame({'fourier_order': fourier_order_list,
                            'rmse': rmse_mean_list,
                            'mape(%)': mape_mean_list,
                            'mae': mae_mean_list})
        
    # Return hyperparam corresponding to lowest error metric
    return fourier_order_list[np.argmin(rmse_mean_list)], results

def hyperparam_tune_wd(df, H, train_size, val_size, window_list, holidays):
    """
    Hyperparameter tuning - upper and lower windows for holidays
    Inputs
        df                     : dataframe with headers 'ds' and 'y' (necessary for Prophet)
        H                      : forecast horizon
        train_size             : length of training set
        val_size               : length of validation set. Note len(df) = train_size + val_size
        window_list            : list of upper and lower window values to try
        holidays               : dataframe containing holidays you will like to model. 
                                 Must have 'holiday' and 'ds' columns
    Outputs
        optimum hyperparameters
    """
    rmse_mean_list = []
    mape_mean_list = []
    mae_mean_list = []
    for window in tqdm_notebook(window_list):
        print("window = " + str(window))
        
        if window is None:
            rmse_mean, mape_mean, mae_mean, _ = get_error_metrics(df=df, 
                                                                  H=H, 
                                                                  train_size=train_size, 
                                                                  val_size=val_size, 
                                                                  holidays=None)
        else:
            # Add lower_window and upper_window which extend the holiday out to 
            # [lower_window, upper_window] days around the date
            holidays['lower_window'] = -window
            holidays['upper_window'] = +window
        
            rmse_mean, mape_mean, mae_mean, _ = get_error_metrics(df=df, 
                                                                  H=H, 
                                                                  train_size=train_size, 
                                                                  val_size=val_size, 
                                                                  holidays=holidays)
        rmse_mean_list.append(rmse_mean)
        mape_mean_list.append(mape_mean)
        mae_mean_list.append(mae_mean)
        
    # Create results dataframe
    results = pd.DataFrame({'window': window_list,
                            'rmse': rmse_mean_list,
                            'mape(%)': mape_mean_list,
                            'mae': mae_mean_list})
        
    # Return hyperparam corresponding to lowest error metric
    return window_list[np.argmin(rmse_mean_list)], results

def hyperparam_tune_cp_fo_wd(df, H, train_size, val_size, changepoint_prior_scale_list, 
                             fourier_order_list, window_list, holidays):
    """
    Hyperparameter tuning - changepoint, fourier_order, holidays
    Inputs
        df                     : dataframe with headers 'ds' and 'y' (necessary for Prophet)
        H                      : forecast horizon
        train_size             : length of training set
        val_size               : length of validation set. Note len(df) = train_size + val_size
        changepoint_prior_scale_list: list of changepoint_prior_scale values to try
        fourier_order_list          : list of fourier_order values to try
        window_list                 : list of upper and lower window values to try
        holidays                    : dataframe containing holidays you will like to model. 
                                      Must have 'holiday' and 'ds' columns
    Outputs
        optimum hyperparameters
    """
    rmse_mean_list = []
    mape_mean_list = []
    mae_mean_list = []
    cp_list = []
    fo_list = []
    wd_list = []
    for changepoint_prior_scale in tqdm_notebook(changepoint_prior_scale_list):
        for fourier_order in tqdm_notebook(fourier_order_list):
            for window in tqdm_notebook(window_list):
                
                if window is None:
                    rmse_mean, mape_mean, mae_mean, _ = get_error_metrics(df, 
                                                                          H, 
                                                                          train_size, 
                                                                          val_size, 
                                                                          changepoint_prior_scale, 
                                                                          fourier_order, 
                                                                          holidays=None)
                else:
                    # Add lower_window and upper_window which extend the holiday out to 
                    # [lower_window, upper_window] days around the date
                    holidays['lower_window'] = -window
                    holidays['upper_window'] = +window
        
                    rmse_mean, mape_mean, mae_mean, _ = get_error_metrics(df, 
                                                                          H, 
                                                                          train_size, 
                                                                          val_size, 
                                                                          changepoint_prior_scale, 
                                                                          fourier_order, 
                                                                          holidays)
                rmse_mean_list.append(rmse_mean)
                mape_mean_list.append(mape_mean)
                mae_mean_list.append(mae_mean)
                cp_list.append(changepoint_prior_scale)
                fo_list.append(fourier_order)
                wd_list.append(window)
        
    # Return hyperparam corresponding to lowest error metric
    results = pd.DataFrame({'changepoint_prior_scale': cp_list, 
                            'fourier_order': fo_list,
                            'window': wd_list,
                            'rmse': rmse_mean_list,
                            'mape(%)': mape_mean_list,
                            'mae': mae_mean_list})
    temp = results[results['rmse'] == results['rmse'].min()]
    changepoint_prior_scale_opt = temp['changepoint_prior_scale'].values[0]
    fourier_order_opt = temp['fourier_order'].values[0]
    window_opt = temp['window'].values[0]
    
    return changepoint_prior_scale_opt, fourier_order_opt, window_opt, results


In [ ]:
def processInput(i, df, H, changepoint_prior_scale, fourier_order, holidays):
    preds_list = get_preds_prophet(df[i-train_size:i], H, changepoint_prior_scale, fourier_order, holidays)
    
    # Compute error metrics
    rmse = get_rmse(df[i:i+H]['y'], preds_list)
    mape = get_mape(df[i:i+H]['y'], preds_list)
    mae = get_mae(df[i:i+H]['y'], preds_list)
    
    return (rmse, mape, mae)

def get_preds_prophet_parallelized(df, H, changepoint_prior_scale=0.05, fourier_order=None, holidays=None):
    """
    This is a parallelized implementation of get_preds_prophet.
    Given a dataframe consisting of both train+validation, do predictions of forecast horizon H on the validation set, 
    at H/2 intervals.
    Inputs
        df                     : dataframe with headers 'ds' and 'y' (necessary for Prophet)
        H                      : forecast horizon
        train_size             : length of training set
        val_size               : length of validation set. Note len(df) = train_size + val_size
        changepoint_prior_scale: to detect changepoints in time series analysis trajectories
        fourier_order          : determines how quickly seasonality can change
        holidays               : dataframe containing holidays you will like to model. 
                                 Must have 'holiday' and 'ds' columns
    Outputs
        mean of rmse, mean of mape, mean of mae, dict of predictions
    """
    inputs = range(train_size, len(df)-H, int(H/2))

    num_cores = multiprocessing.cpu_count()

    results = Parallel(n_jobs=num_cores)(delayed(processInput)(i, df, H, changepoint_prior_scale, fourier_order, holidays) for i in inputs)
    # results has format [(rmse1, mape1, mae1), (rmse2, mape2, mae2), ...]

    rmse = [errors[0] for errors in results]
    mape = [errors[1] for errors in results]
    mae = [errors[2] for errors in results]
    
    return np.mean(rmse), np.mean(mape), np.mean(mae)


## Load data


In [ ]:
df = pd.read_csv(stk_path, sep = ",")

# Convert Date column to datetime
df.loc[:, 'Date'] = pd.to_datetime(df['Date'],format='%Y-%m-%d')

# Change all column headings to be lower case, and remove spacing
df.columns = [str(x).lower().replace(' ', '_') for x in df.columns]

# Sort by datetime
df.sort_values(by='date', inplace=True, ascending=True)

# Get size of dataframe
print("len(df) = " + str(len(df)))

df.head(10)


In [ ]:
df['date'].min(), df['date'].max()

In [ ]:
# Plot adjusted close over time
rcParams['figure.figsize'] = 10, 8 # width 10, height 8

ax = df.plot(x='date', y='adj_close', style='b-', grid=True)
ax.set_xlabel("date")
ax.set_ylabel("USD")

In [ ]:
# Prepare dataframe for Prophet
df_prophet = df[['date', 'adj_close']].rename(columns={'date':'ds', 'adj_close':'y'})
df_prophet.head()

## Predict for a specific H (forecast horizon) and a specific date


In [ ]:
i = train_val_size # Predict for day i, for the next H-1 days. Note indexing of days start from 0.
print("Predicting on day %d, date %s, with forecast horizon H = %d" % (i, df_prophet.iloc[i]['ds'], H))

In [ ]:
# Fit prophet model
m = Prophet(daily_seasonality=True)
m.fit(df_prophet[i-train_val_size:i],)

In [ ]:

# Create dataframe with the dates we want to predict
future = m.make_future_dataframe(periods=2*H)

In [ ]:
# Eliminate weekend from future dataframe
future['day'] = future['ds'].dt.weekday
future = future[future['day']<=4]

# Predict
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
# Plot
m.plot(forecast);

In [ ]:
# Plot components
m.plot_components(forecast);

In [ ]:
    # Compute error metrics
    
preds_list = forecast['yhat'][train_val_size:train_val_size+H]
print("For forecast horizon %d, predicting on day %d, date %s, the RMSE is %f" % (H, i, df['date'][i],
                                                                                  get_rmse(df_prophet[i:i+H]['y'],
                                                                                           preds_list)))
print("For forecast horizon %d, predicting on day %d, date %s, the mean MAPE is %f" % (H, i, df['date'][i],
                                                                                       get_mape(df_prophet[i:i+H]['y'],
                                                                                                preds_list)))
print("For forecast horizon %d, predicting on day %d, date %s, the mean MAE is %f" % (H, i, df['date'][i],
                                                                                      get_mae(df_prophet[i:i+H]['y'],
                                                                                              preds_list)))

In [ ]:
# Plot the predictions
rcParams['figure.figsize'] = 10, 8 # width 10, height 8
matplotlib.rcParams.update({'font.size': 14})

ax = df.plot(x='date', y='adj_close', style='bx-', grid=True)

# Plot the predictions
ax.plot(df['date'][i:i+H], preds_list, marker='x')
    
ax.set_xlabel("date")
ax.set_ylabel("USD")
ax.legend(['adj_close', 'predictions'])
ax.set_ylim([105, 120])
ax.set_xlim([date(2016, 11, 1), date(2017, 2, 28)])

## Predict for a specific H (forecast horizon) and a specific date, with hyperparameter tuning - changepoint

In [ ]:
i = train_val_size # Predict for day i, for the next H-1 days. Note indexing of days start from 0.
print("Predicting on day %d, date %s, with forecast horizon H = %d" % (i, df_prophet.iloc[i]['ds'], H))

In [ ]:
# Get optimum hyperparams
tic = time.time()
changepoint_opt, results = hyperparam_tune_cp(df_prophet[i-train_val_size:i], 
                                              H, 
                                              train_size, 
                                              val_size, 
                                              changepoint_prior_scale_list)
toc = time.time()
print("Time taken = " + str((toc-tic)/60.0) + " mins")

print("changepoint_opt = " + str(changepoint_opt))

results

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Get predictions using tuned hyperparams
preds_list = get_preds_prophet(df_prophet[i-train_val_size:i], H, changepoint_prior_scale=changepoint_opt, )
print("For forecast horizon %d, predicting on day %d, date %s, the RMSE is %f" % (H, i, df['date'][i],
                                                                                  get_rmse(df_prophet[i:i+H]['y'], 
                                                                                           preds_list)))
print('')
print("For forecast horizon %d, predicting on day %d, date %s, the mean MAPE is %f" % (H, i, df['date'][i], 
                                                                                       get_mape(df_prophet[i:i+H]['y'],
                                                                                                preds_list)))
print('')
print("For forecast horizon %d, predicting on day %d, date %s, the mean MAE is %f" % (H, i, df['date'][i],
                                                                                      get_mae(df_prophet[i:i+H]['y'],
                                                                                              preds_list)))

In [ ]:
# Plot the predictions
rcParams['figure.figsize'] = 10, 8 # width 10, height 8

ax = df.plot(x='date', y='adj_close', style='bx-', grid=True)

# Plot the predictions
ax.plot(df['date'][i:i+H], preds_list, marker='x')
    
ax.set_xlabel("date")
ax.set_ylabel("USD")
ax.legend(['adj_close', 'predictions'])
ax.set_ylim([105, 120])
ax.set_xlim([date(2016, 11, 1), date(2017, 2, 28)])

## Predict for a specific H (forecast horizon) and a specific date, with hyperparameter tuning - monthly seasonality

In [ ]:
i = train_val_size # Predict for day i, for the next H-1 days. Note indexing of days start from 0.
print("Predicting on day %d, date %s, with forecast horizon H = %d" % (i, df_prophet.iloc[i]['ds'], H))


In [ ]:
# Get optimum hyperparams
tic = time.time()
fourier_order_opt, results = hyperparam_tune_fo(df_prophet[i-train_val_size:i], 
                                                H, 
                                                train_size, 
                                                val_size, 
                                                fourier_order_list)
toc = time.time()
print("Time taken = " + str((toc-tic)/60.0) + " mins")

print("fourier_order_opt = " + str(fourier_order_opt))

results

In [ ]:
# Get predictions using tuned hyperparams
preds_list = get_preds_prophet(df_prophet[i-train_val_size:i], H, fourier_order=fourier_order_opt)
print("For forecast horizon %d, predicting on day %d, date %s, the RMSE is %f" % (H, i, df['date'][i], get_rmse(df_prophet[i:i+H]['y'], preds_list)))
print("For forecast horizon %d, predicting on day %d, date %s, the mean MAPE is %f" % (H, i, df['date'][i], get_mape(df_prophet[i:i+H]['y'], preds_list)))
print("For forecast horizon %d, predicting on day %d, date %s, the mean MAE is %f" % (H, i, df['date'][i], get_mae(df_prophet[i:i+H]['y'], preds_list)))


In [ ]:
# Plot the predictions
rcParams['figure.figsize'] = 10, 8 # width 10, height 8

ax = df.plot(x='date', y='adj_close', style='bx-', grid=True)

# Plot the predictions
ax.plot(df['date'][i:i+H], preds_list, marker='x')
    
ax.set_xlabel("date")
ax.set_ylabel("USD")
ax.legend(['adj_close', 'predictions'])
ax.set_ylim([105, 120])
ax.set_xlim([date(2016, 11, 1), date(2017, 2, 28)])

## Predict for a specific H (forecast horizon) and a specific date, with hyperparameter tuning - holidays


In [ ]:
i = train_val_size # Predict for day i, for the next H-1 days. Note indexing of days start from 0.
print("Predicting on day %d, date %s, with forecast horizon H = %d" % (i, df_prophet.iloc[i]['ds'], H))

In [ ]:
# Load holidays file
hols = pd.read_csv(holidays_path, header=None, sep="\t", names=['ds', 'holiday'])

# Convert Date column to datetime
hols.loc[:, 'ds'] = pd.to_datetime(hols['ds'],format='%Y-%m-%d')

# Add lower_window and upper_window which extend the holiday out to [lower_window, upper_window] days around the date
hols['lower_window'] = 0
hols['upper_window'] = 0

hols.head()

In [ ]:

# Get optimum hyperparams
tic = time.time()
window_opt, results = hyperparam_tune_wd(df_prophet[i-train_val_size:i], 
                                         H, 
                                         train_size, 
                                         val_size, 
                                         window_list,
                                         hols)
toc = time.time()
print("Time taken = " + str((toc-tic)/60.0) + " mins")

print("window_opt = " + str(window_opt))

results

In [ ]:
# Get predictions using tuned hyperparams
if window_opt is None:
    preds_list = get_preds_prophet(df_prophet[i-train_val_size:i], H, holidays=None)
else:
    hols['lower_window'] = -window_opt
    hols['upper_window'] = +window_opt
    preds_list = get_preds_prophet(df_prophet[i-train_val_size:i], H, holidays=hols)
print("For forecast horizon %d, predicting on day %d, date %s, the RMSE is %f" % (H, i, df['date'][i], get_rmse(df_prophet[i:i+H]['y'], preds_list)))
print("For forecast horizon %d, predicting on day %d, date %s, the mean MAPE is %f" % (H, i, df['date'][i], get_mape(df_prophet[i:i+H]['y'], preds_list)))
print("For forecast horizon %d, predicting on day %d, date %s, the mean MAE is %f" % (H, i, df['date'][i], get_mae(df_prophet[i:i+H]['y'], preds_list)))

In [ ]:
# Plot the predictions
rcParams['figure.figsize'] = 10, 8 # width 10, height 8

ax = df.plot(x='date', y='adj_close', style='bx-', grid=True)

# Plot the predictions
ax.plot(df['date'][i:i+H], preds_list, marker='x')
    
ax.set_xlabel("date")
ax.set_ylabel("USD")
ax.legend(['adj_close', 'predictions'])
ax.set_ylim([105, 120])
ax.set_xlim([date(2016, 11, 1), date(2017, 2, 28)])

## Predict for a specific H (forecast horizon) and a specific date, with hyperparameter tuning - changepoint, monthly seasonality, holidays

In [ ]:
i = train_val_size # Predict for day i, for the next H-1 days. Note indexing of days start from 0.
print("Predicting on day %d, date %s, with forecast horizon H = %d" % (i, df_prophet.iloc[i]['ds'], H))

In [ ]:
# Get optimum hyperparams
tic = time.time()
changepoint_prior_scale_opt, fourier_order_opt, window_opt, results = \
hyperparam_tune_cp_fo_wd(df_prophet[i-train_val_size:i], 
                         H, 
                         train_size, 
                         val_size, 
                         changepoint_prior_scale_list,
                         fourier_order_list,
                         window_list,
                         hols)

toc = time.time()

results

In [ ]:
print("Time taken = " + str((toc-tic)/60.0) + " mins")

print("changepoint_prior_scale_opt = " + str(changepoint_prior_scale_opt))
print("fourier_order_opt = " + str(fourier_order_opt))
print("window_opt = " + str(window_opt))

In [ ]:
# Print the tail of the results df
results.tail()

In [ ]:
# Plot the predictions
rcParams['figure.figsize'] = 10, 8 # width 10, height 8

ax = df.plot(x='date', y='adj_close', style='bx-', grid=True)

# Plot the predictions
ax.plot(df['date'][i:i+H], preds_list, marker='x')
    
ax.set_xlabel("date")
ax.set_ylabel("USD")
ax.legend(['adj_close', 'predictions'])
ax.set_ylim([105, 120])
ax.set_xlim([date(2016, 11, 1), date(2017, 2, 28)])

## Predict for a specific H (forecast horizon), and various dates, using model trained in previous step

In [ ]:
changepoint_prior_scale_opt = 2.5
fourier_order_opt = 10
window_opt = None

In [ ]:
# Predict using Prophet, and compute error metrics also
rmse = [] # root mean square error
mape = [] # mean absolute percentage error
mae = []  # mean absolute error
preds_dict = {}
H = 21 # We will test with this value of H and observe the results
i_list = range(train_val_size, train_val_size+84*5+42+1, 42)
for i in i_list:
# for i in tqdm_notebook(range(train_val_size, len(df)-H, int(H/2))): # Do a forecast on day i
    print("Predicting on day %d, date %s" % (i, df_prophet['ds'][i]))
    
    # Get predictions using tuned hyperparams
    if (window_opt is None) or (np.isnan(window_opt)):
        preds_list = get_preds_prophet(df_prophet[i-train_val_size:i], 
                                       H, 
                                       changepoint_prior_scale_opt, 
                                       fourier_order_opt, 
                                       holidays=None)
    else:
        hols['lower_window'] = -window_opt
        hols['upper_window'] = +window_opt
        preds_list = get_preds_prophet(df_prophet[i-train_val_size:i], 
                                       H, 
                                       changepoint_prior_scale_opt, 
                                       fourier_order_opt, 
                                       holidays=hols)
    
    # Collect the predictions
    preds_dict[i] = preds_list
    
    # Compute error metrics
    rmse.append(get_rmse(df_prophet[i:i+H]['y'], preds_list))
    mape.append(get_mape(df_prophet[i:i+H]['y'], preds_list))
    mae.append(get_mae(df_prophet[i:i+H]['y'], preds_list))

print("Altogether we made %d forecasts, each of length %d days" % (len(rmse), H))


In [ ]:
print("For forecast horizon %d, the mean RMSE is %f" % (H, np.mean(rmse)))
print("For forecast horizon %d, the mean MAPE is %f" % (H, np.mean(mape)))
print("For forecast horizon %d, the mean MAE is %f" % (H, np.mean(mae)))

In [ ]:
results_final_no_tune = pd.DataFrame({'day': i_list,
                                      'changepoint_opt': [changepoint_prior_scale_opt]*len(i_list),
                                      'fourier_order_opt': [fourier_order_opt]*len(i_list),
                                      'window_opt': [window_opt]*len(i_list),
                                      'rmse': rmse,
                                      'mape': mape,
                                      'mae': mae})
results_final_no_tune

In [ ]:
# Plot the predictions
rcParams['figure.figsize'] = 10, 8 # width 10, height 8

ax = df.plot(x='date', y='adj_close', style='b-', grid=True)

# Plot the predictions
for key in preds_dict:
    ax.plot(df['date'][key:key+H], preds_dict[key])
    
ax.set_xlabel("date")
ax.set_ylabel("USD")

In [ ]:

# Plot the predictions, and zoom in
rcParams['figure.figsize'] = 10, 8 # width 10, height 8

ax = df.plot(x='date', y='adj_close', style='b-', grid=True)

# Plot the predictions
for key in preds_dict:
    ax.plot(df['date'][key:key+H], preds_dict[key])
    
ax.set_xlabel("date")
ax.set_ylabel("USD")
ax.legend(['adj_close', 'predictions'])
ax.set_ylim([105, 150])
ax.set_xlim([date(2017, 1, 1), date(2018, 12, 31)])

## Predict for a specific H (forecast horizon), and various dates, tuning model for every prediction

In [ ]:
i_list = [train_val_size+294, train_val_size+378, train_val_size+462]
for i in i_list:
    print("Predicting on day %d, date %s, with forecast horizon H = %d" % (i, df_prophet.iloc[i]['ds'], H))

    # Predict using Prophet, and compute error metrics also
    rmse = [] # root mean square error
    mape = [] # mean absolute percentage error
    mae = []  # mean absolute error
    preds_dict = {}

    tic = time.time()
    
    changepoint_prior_scale_opt, fourier_order_opt, window_opt, results = \
        hyperparam_tune_cp_fo_wd(df_prophet[i-train_val_size:i], 
                                 H, 
                                 train_size, 
                                 val_size, 
                                 changepoint_prior_scale_list,
                                 fourier_order_list,
                                 window_list,
                                 hols)
    
    # Get predictions using tuned hyperparams
    if (window_opt is None) or (np.isnan(window_opt)):
        preds_list = get_preds_prophet(df_prophet[i-train_val_size:i], 
                                       H, 
                                       changepoint_prior_scale_opt, 
                                       fourier_order_opt, 
                                       holidays=None)
    else:
        hols['lower_window'] = -window_opt
        hols['upper_window'] = +window_opt
        preds_list = get_preds_prophet(df_prophet[i-train_val_size:i], 
                                       H, 
                                       changepoint_prior_scale_opt, 
                                       fourier_order_opt, 
                                       holidays=hols)
    
    # Compute error metrics
    rmse = get_rmse(df_prophet[i:i+H]['y'], preds_list)
    mape = get_mape(df_prophet[i:i+H]['y'], preds_list)
    mae = get_mae(df_prophet[i:i+H]['y'], preds_list)

    # Put results into pickle
    pickle.dump(preds_list, open("./data/preds_list_" + str(i) + ".pickle", "wb"))
    pickle.dump(rmse, open("./data/rmse_" + str(i) + ".pickle", "wb"))
    pickle.dump(mape, open("./data/mape_" + str(i) + ".pickle", "wb"))
    pickle.dump(mae, open("./data/mae_" + str(i) + ".pickle", "wb"))
    pickle.dump(changepoint_prior_scale_opt, open("./data/changepoint_prior_scale_opt_" + str(i) + ".pickle", "wb"))
    pickle.dump(fourier_order_opt, open("./data/fourier_order_opt_" + str(i) + ".pickle", "wb"))
    pickle.dump(window_opt, open("./data/window_opt_" + str(i) + ".pickle", "wb"))
    pickle.dump(results, open("./data/results_" + str(i) + ".pickle", "wb"))
    
    toc = time.time()
    print("Time taken = " + str((toc-tic)/60.0))


In [ ]:
# Load pickle file
i_list = range(train_val_size, train_val_size+84*5+42+1, 42)
# i_list = [train_val_size, train_val_size+42, train_val_size+84, train_val_size+126, train_val_size+84*2, 
#           train_val_size+210, train_val_size+84*3, train_val_size+84*4, train_val_size+84*5]
preds_dict = {}
rmse, mape, mae, changepoint_prior_scale_opt, fourier_order_opt, window_opt = [], [], [], [], [], []

for i in i_list:
    preds_dict[i] = pickle.load(open( "./data/preds_list_" + str(i) + ".pickle", "rb"))
    rmse.append(pickle.load(open( "./data/rmse_" + str(i) + ".pickle", "rb")))
    mape.append(pickle.load(open( "./data/mape_" + str(i) + ".pickle", "rb")))
    mae.append(pickle.load(open( "./data/mae_" + str(i) + ".pickle", "rb")))
    changepoint_prior_scale_opt.append(pickle.load(open( "./data/changepoint_prior_scale_opt_" + str(i) + ".pickle", "rb")))
    fourier_order_opt.append(pickle.load(open( "./data/fourier_order_opt_" + str(i) + ".pickle", "rb")))
    window_opt.append(pickle.load(open( "./data/window_opt_" + str(i) + ".pickle", "rb")))


In [ ]:
print("For forecast horizon %d, the mean RMSE is %f" % (H, np.mean(rmse)))
print("For forecast horizon %d, the mean MAPE is %f" % (H, np.mean(mape)))
print("For forecast horizon %d, the mean MAE is %f" % (H, np.mean(mae)))


In [ ]:
results_final = pd.DataFrame({'day': i_list,
                              'changepoint_opt': changepoint_prior_scale_opt,
                              'fourier_order_opt': fourier_order_opt,
                              'window_opt': window_opt,
                              'rmse': rmse,
                              'mape': mape,
                              'mae': mae})
results_final

In [ ]:
# Plot the predictions, and zoom in
rcParams['figure.figsize'] = 10, 8 # width 10, height 8

ax = df.plot(x='date', y='adj_close', style='b-', grid=True)

# Plot the predictions
for key in preds_dict:
    ax.plot(df['date'][key:key+H], preds_dict[key])
    
ax.set_xlabel("date")
ax.set_ylabel("USD")
ax.set_ylim([105, 150])
ax.set_xlim([date(2017, 1, 1), date(2018, 12, 31)])

## Findings

In [ ]:
# Consolidate results from hyperparameter tuning
pd.set_option('display.max_colwidth', -1)  # or 199
results = pd.DataFrame({'Method': ['Last Value', 
                                   'Prophet - no hyperparameter tuning', 
                                   'Prophet tuning changepoint', 
                                   'Prophet tuning fourier order', 
                                   'Prophet tuning window size', 
                                   'Prophet tuning changepoint, fourier order, and window size'],
                        'RMSE'    : [2.56, 3.40, 0.64, 3.52, 3.40, 0.55],
                        'MAPE(%)' : [1.65, 2.87, 0.52, 2.98, 2.87, 0.41],
                        'MAE'     : [2.19, 3.25, 0.58, 3.37, 3.25, 0.47]                       
                       })
results